<a href="https://colab.research.google.com/github/MGASALUCAS/Notebooks-Collection/blob/main/ZindiChallenge1(Soln2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [3]:
import numpy as np
import pandas as pd

import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [5]:
Train = pd.read_csv("Train.csv")
Test = pd.read_csv("Test.csv")

In [6]:
#what features are categorical?
categorical_features = Train.select_dtypes(include=['object','category']).columns.tolist()

#what features are numerical?
numerical_features = Train.select_dtypes(include=['int64','float64']).columns.tolist()
print("categorical features", categorical_features)
print()
print("numerical features",numerical_features)

categorical features ['user_id', 'REGION', 'TENURE', 'MRG', 'TOP_PACK']

numerical features ['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'ZONE1', 'ZONE2', 'REGULARITY', 'FREQ_TOP_PACK', 'CHURN']


In [7]:
defective_features = ['user_id', 'MRG',"TOP_PACK"]
Train.drop(defective_features, 1, inplace = True)
Test.drop(defective_features, 1, inplace = True)

In [8]:
ntrain = Train.shape[0]
ntest = Test.shape[0]
data = pd.concat((Train, Test)).reset_index(drop=True)

In [9]:
data['TENURE_avg'] = data['TENURE'].map({'K > 24 month': (24+27)/2, 'I 18-21 month':(18+21)/2 , 'H 15-18 month': (15+18)/2, 'G 12-15 month':(12+15)/2,
                                             'J 21-24 month': (21+24)/2, 'F 9-12': (9+12)/2, 'E 6-9 month':(6+9)/2, 'D 3-6 month':(3+6)/2})

data['TENURE'] = data['TENURE'].map({'K > 24 month': 24, 'I 18-21 month': 18, 'H 15-18 month': 15, 'G 12-15 month':12,
                                             'J 21-24 month': 21, 'F 9-12': 9, 'E 6-9 month':6, 'D 3-6 month':3})

In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['REGION_encoded']=le.fit_transform(data['REGION'])
data.drop(['REGION'],1,inplace=True)

In [11]:
data['Total_income'] = data['REVENUE'] * data['FREQUENCE']
data['FREQ_PACK'] = data.FREQUENCE_RECH/data.FREQ_TOP_PACK
data['diff//freq'] = (data['MONTANT'] - data['FREQUENCE_RECH']) / data['FREQUENCE']
data['NOT_FREQUENCE_RECH'] = data['FREQUENCE_RECH'] - data['FREQ_TOP_PACK']

In [12]:
data['diff_Orange'] = np.abs(data['ON_NET']-data['ORANGE'])
data['diff_Tigo'] = np.abs(data['ON_NET']-data['TIGO'])
data['freq//rech'] = data['FREQUENCE'] / data['FREQUENCE_RECH']
data['freq//montant'] =  data['MONTANT']/ data['FREQUENCE']
data['freq//revenue'] = data['FREQUENCE'] / data['REVENUE']

In [13]:
data['segment/reg'] = data['ARPU_SEGMENT'] / data['REGULARITY']
data['net//reg'] = data['ON_NET'] / data['REGULARITY']
data['data//reg'] = data['DATA_VOLUME'] / data['REGULARITY']

In [14]:
train_data = data[:ntrain]
test_data = data[ntrain:]
target = train_data['CHURN']
train_data.drop(["CHURN"],1,inplace=True)
test_data.drop(["CHURN"],1,inplace=True)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [16]:
#Segregate data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size=0.33, stratify=target,random_state=56)
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

ValueError: ignored

In [ ]:
from sklearn.metrics import log_loss,roc_auc_score
import lightgbm as lgbm
import xgboost as xgb
from catboost import CatBoostClassifier
import lightgbm as lgbm

In [ ]:
def predict(estimator,data):
    predictions = estimator.predict_proba(data)[:,1]
    return predictions

In [ ]:
#Initialize models and their hyper-parameters
xgb_model = xgb.XGBClassifier(n_jobs=-1,random_state=23,objective='binary:logistic',
                n_estimators=2500,learning_rate=0.01,
                colsample_bytree=0.9,subsample=1, use_label_encoder=False)
cat_model = CatBoostClassifier(random_seed=34,use_best_model=True,
                          n_estimators=5000,silent=True,eval_metric='Logloss')
lgb = lgbm.LGBMClassifier(random_state=734, n_estimators=5000,num_leaves=120,learning_rate=0.008,
    max_depth=9)

In [ ]:
#Define training loop
def train(estimator):
    fold=0
    scores,pp=[],[]
    for train_index, test_index in skf.split(X_train,y_train):
        fold+=1
        print(f"================Fold:{fold}====================")
        xtrain, xtest = X_train.iloc[train_index],X_train.iloc[test_index]
        ytrain, ytest = y_train.iloc[train_index],y_train.iloc[test_index]
        model = estimator.fit(xtrain,ytrain,early_stopping_rounds=200,eval_metric="logloss",
                      eval_set=[(xtest,ytest)],verbose=250)
        prediction = model.predict_proba(xtest)
        predict_ = model.predict_proba(test_data)
        score = log_loss(ytest,prediction)
        print(f"Log loss for {fold}: {score}")
        scores.append(score)
        pp.append(predict_)

        #0.2528473
      #Baseline Mean: 0.25479403  
    print(f"The Mean Log_loss eror: {np.mean(scores)}")##0.252703
    return pp

#Train Xgboost model
xgb_pred = train(xgb_model)

In [ ]:
#Train LightGBM model
lgb_pred = train(lgb)

In [ ]:
#Define training loop
fold=0
scores,pp=[],[]
estimator=cat_model
for train_index, test_index in skf.split(X_train,y_train):
    fold+=1
    print(f"================Fold:{fold}====================")
    xtrain, xtest = X_train.iloc[train_index],X_train.iloc[test_index]
    ytrain, ytest = y_train.iloc[train_index],y_train.iloc[test_index]
    model = estimator.fit(xtrain,ytrain,early_stopping_rounds=200,
                  eval_set=[(xtest,ytest)],verbose=250)
    prediction = model.predict_proba(xtest)
    predict_ = model.predict_proba(test_data)
    score = log_loss(ytest,prediction)
    print(f"Log loss for {fold}: {score}")
    scores.append(score)
    pp.append(predict_)

  
print(f"The Mean Log_loss eror: {np.mean(scores)}")##0.252703
cat_predict=pp

In [ ]:
#Aggregate predictions for all three models
cat_pred = np.mean(cat_predict,0)[:,1]
lgb_pred = np.mean(lgb_pred,0)[:,1]
xgb_pred = np.mean(xgb_pred,0)[:,1]

In [ ]:
#For better perfromance a blend of all three model's prediction is required.
#Weights (e.g 0.6) are assigned on the basis of individual perfromance.
predictions=(((0.6*cat_pred) + (0.3*lgb_pred) + (0.1*cat_pred)) + ((0.45*cat_pred) + (0.55*lgb_pred)))/2

In [ ]:
#ubmit predictions
ss= pd.read_csv("SampleSubmission.csv")
ss['CHURN'] = predictions
ss.to_csv("Submission.csv",index=False)

In [ ]:
#SECOND NOTEBOOK SOLUTION;

In [17]:
!pip install catboost 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cat_
import seaborn as sns
import lightgbm as lgb

In [19]:
class func() :   
    def __init__(self, train, label, test, model, model_type, random_state):
        self.train, self.label, self.test = train, label, test
        self.model, self.model_type = model, model_type
        self.random_state = random_state
        
        assert self.model_type in ('catboost', 'xgboost', 'lgbm'), 'Incorrect model_type'
    def __call__(self, plot = True):
        return self.fit(plot)

    def fit(self, plot):
        def catboost_fit(X_train, X_test, y_train, y_test):
            self.model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=500,
                           verbose=50,use_best_model=True)
            x_test_predict = self.model.predict_proba(X_test)[:,1]
            x_train_predict = self.model.predict_proba(X_train)[:,1]
            self.val_p[test_index] = x_test_predict
            self.test_p += self.model.predict_proba(self.test)[:,1]
            return x_test_predict, x_train_predict

        def xgboost_fit(X_train, X_test, y_train, y_test):
            self.model.fit(X_train, y_train, early_stopping_rounds = 30, eval_metric="auc",
                           eval_set=[(X_test, y_test)], verbose = True)
            x_test_predict = self.model.predict_proba(X_test, ntree_limit = self.model.get_booster().best_ntree_limit)[:,1]
            x_train_predict = self.model.predict_proba(X_train, ntree_limit = self.model.get_booster().best_ntree_limit)[:,1]
            self.val_p[test_index] = x_test_predict
            self.test_p += self.model.predict_proba(self.test, ntree_limit = self.model.get_booster().best_ntree_limit)[:,1]
            return x_test_predict, x_train_predict

        def lgbm_fit(X_train, X_test, y_train, y_test):
            self.model.fit(X_train, y_train, early_stopping_rounds = 30, eval_metric="auc",
                           eval_set=[(X_test, y_test)], verbose = True)
            x_test_predict = self.model.predict_proba(X_test, num_iteration = self.model.best_iteration_)[:,1]
            x_train_predict = self.model.predict_proba(X_train, num_iteration = self.model.best_iteration_)[:,1]
            self.val_p[test_index] = x_test_predict
            self.test_p += self.model.predict_proba(self.test, num_iteration = self.model.best_iteration_)[:,1]
            return x_test_predict, x_train_predict


        self.val_p = np.zeros(self.train.shape[0])
        mean_val = []
        mean_train = []
        self.test_p = np.zeros(self.test.shape[0])
        splits = 5
        kf = StratifiedKFold(n_splits = splits)
        for fold_count, (train_index, test_index) in enumerate(kf.split(self.train, self.label)):
            X_train,X_test = self.train.iloc[train_index],self.train.iloc[test_index]
            y_train,y_test = self.label.iloc[train_index],self.label.iloc[test_index]

            print(f"================================Fold{fold_count+1}====================================")
            if self.model_type == 'catboost': x_test_predict, x_train_predict = catboost_fit(X_train, X_test, y_train, y_test)
            elif self.model_type == 'xgboost': x_test_predict, x_train_predict = xgboost_fit(X_train, X_test, y_train, y_test)
            elif self.model_type == 'lgbm': x_test_predict, x_train_predict = lgbm_fit(X_train, X_test, y_train, y_test)

            print('\nValidation scores', roc_auc_score(y_test, x_test_predict), log_loss(y_test, x_test_predict))
            print('Training scores', roc_auc_score(y_train, x_train_predict), log_loss(y_train, x_train_predict))
            mean_val.append(roc_auc_score(y_test, x_test_predict))
            mean_train.append(roc_auc_score(y_train, x_train_predict))

        if plot:
            feat_imp = pd.DataFrame(sorted(zip(self.model.feature_importances_,self.train.columns)), columns=['Value','Feature'])
            plt.figure(figsize=(30,25))
            sns.barplot(x="Value", y="Feature", data=feat_imp.sort_values(by="Value", ascending=False))
            plt.ylabel('Feature Importance Score')
            plt.show()
        print(np.mean(mean_val), np.mean(mean_train), np.std(mean_val))
        return self.val_p, self.test_p/splits, self.model

In [20]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
submission = pd.read_csv('SampleSubmission.csv')

In [21]:
print(train.MRG.value_counts(dropna = False), 'Train length:', len(train), '\n')
print(test.MRG.value_counts(dropna = False), 'Test length:', len(test))

NO     34888
NaN        1
Name: MRG, dtype: int64 Train length: 34889 

NO     53135
NaN        1
Name: MRG, dtype: int64 Test length: 53136


In [22]:
train.drop(['user_id', 'MRG',], 1, inplace = True)
test.drop(['user_id', 'MRG',], 1, inplace = True)

In [23]:
ntrain = train.shape[0]
ntest = test.shape[0]
data = pd.concat((train, test)).reset_index(drop=True)

In [24]:
data['REVENUE-MONTANT'] = data['REVENUE'] - data['MONTANT']
data['REVENUE/MONTANT'] = data['REVENUE'] / data['MONTANT']

In [25]:
data['TENURE'] = data['TENURE'].map({'K > 24 month': 24, 'I 18-21 month': 18, 'H 15-18 month': 15, 'G 12-15 month':12,
                                             'J 21-24 month': 21, 'F 9-12': 9, 'E 6-9 month':6, 'D 3-6 month':3})

In [26]:
#RETURN THE AVERAGE OF THE MONTH/TENURE BOUNDARIES
data['TENURE_avg'] = data['TENURE'].map({'K > 24 month': (24+27)/2, 'I 18-21 month':(18+21)/2 , 'H 15-18 month': (15+18)/2, 'G 12-15 month':(12+15)/2,
                                             'J 21-24 month': (21+24)/2, 'F 9-12': (9+12)/2, 'E 6-9 month':(6+9)/2, 'D 3-6 month':(3+6)/2})

In [27]:
data['TENURE/FREQUENCE_RECH'] = data['TENURE_avg'] / data['FREQUENCE_RECH']
data['TENURE/REGULARITY'] = data['TENURE_avg'] / data['REGULARITY']

In [28]:
region = data.groupby('REGION').mean()
region.drop('CHURN', 1, inplace = True)
cols = []
for i in region.columns:
    if i != 'REGION':
        region[i+'_reg_mean_all'] = region[i]
        region.drop(i, 1, inplace = True)
        cols.append(i+'_reg_mean_all')

data = pd.merge(data, region, on='REGION', how = 'left')
for col in cols: data[col+'_freq'] = data[col].map(data[col].value_counts().to_dict())/len(data)

In [29]:
data['freq*montant'] = data['FREQUENCE'] * data['MONTANT']
data['freq*rech'] = data['FREQUENCE'] * data['FREQUENCE_RECH']
data['freq*revenue'] = data['FREQUENCE'] * data['REVENUE']
data['freq*segment'] = data['FREQUENCE'] * data['ARPU_SEGMENT']

data['freq/montant'] =  data['MONTANT']/ data['FREQUENCE']
data['freq/rech'] = data['FREQUENCE'] / data['FREQUENCE_RECH']
data['freq/revenue'] = data['FREQUENCE'] / data['REVENUE']
data['freq/segment'] = data['FREQUENCE'] / data['ARPU_SEGMENT']

In [30]:
data['data/reg'] = data['DATA_VOLUME'] / data['REGULARITY']
data['net/reg'] = data['ON_NET'] / data['REGULARITY']
data['montant-rech/freq'] = (data['MONTANT'] - data['FREQUENCE_RECH']) / data['FREQUENCE']
data['segment/reg'] = data['ARPU_SEGMENT'] / data['REGULARITY']

In [31]:
data['reg_log'] = np.log1p(data['REGULARITY'])
data['rech_log'] = np.log1p(data['FREQUENCE_RECH'])
data['data_log'] = np.log1p(data['DATA_VOLUME'])
data['montant_log'] = np.log1p(data['MONTANT'])
data['rev_log'] = np.log1p(data['REVENUE'])
data['freq_log'] = np.log1p(data['FREQUENCE'])

In [32]:
data['reglog-montlog'] = data['reg_log'] - data['montant_log']
data['revlog/montlog'] = data['REVENUE'] / data['montant_log']
data['tenure/rechlog'] = data['TENURE_avg'] / data['rech_log']
data['reglog-datalog'] = data['reg_log'] - data['data_log']

In [33]:
drop = ['REGION', 'TOP_PACK']
data.drop(drop, 1, inplace = True)

In [37]:
train = data[:ntrain].copy()
#train.drop_duplicates(inplace = True, ignore_index=True)
target = train.CHURN.copy()
train.drop('CHURN', axis=1, inplace=True)

test = data[ntrain:].copy()
test.drop('CHURN', axis=1, inplace=True)
test = test.reset_index(drop=True)

In [38]:
catboost = cat_.CatBoostClassifier(n_estimators=10000, max_depth=6, eval_metric='AUC', reg_lambda = 370, task_type="GPU", devices='0:1')

func_= func(train, target, test, catboost, 'catboost', 1000)
val_p1, test_p1, model1 = func_()

ValueError: ignored

In [39]:
xgboost = xgb.XGBClassifier(objective ='binary:logistic', 
                          eta = 0.99,
                          max_depth = 6, 
                          n_estimators = 5000,
                          reg_lambda = 500,
                          sub_sample = 0.8,
                          colsample_bytree = 0.8, tree_method='gpu_hist', gpu_id=0)

func_= func(train, target, test, xgboost, 'xgboost', 1000)
val_p2, test_p2, model2 = func_()

ValueError: ignored

In [40]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(objective =  'binary', 
                            metric= 'auc',
                            boosting_type= 'gbdt',
                            lambda_l1= 0.0004912993970392775,
                            lambda_l2= 9.424350138808432,
                            num_leaves= 24,
                            feature_fraction= 1.0,
                            bagging_fraction= 0.9540416539312312,
                            bagging_freq= 7,
                            min_child_samples= 100, n_estimators = 300)

func_= func(train, target, test, lgb_model, 'lgbm', 1000)
val_p3, test_p3, model3 = func_()

ValueError: ignored

In [35]:
from sklearn.linear_model import  LinearRegression, Ridge, Lasso
stack = np.column_stack((val_p1, val_p2, val_p3))
stack_p = np.column_stack((test_p1, test_p2, test_p3))
predict = LinearRegression().fit(stack, target).predict(stack_p)

NameError: ignored

In [41]:
submission['CHURN'] = predict

submission.to_csv('solution_solution.csv',index=False)

NameError: ignored